# `concurrent.futures` Executor

In [1]:
%run -m literary.notebook

import time

In [2]:
import asyncio
import asyncio.futures
import weakref
from concurrent import futures

from .executor import AsyncExecutor
from .futures import chain_future_exception, chain_future_handle, create_future

The `ConcurrentFuturesExecutor` accepts a `futures.Executor` object:

In [3]:
class ConcurrentFuturesExecutor(AsyncExecutor):
    def __init__(self, executor: futures.Executor):
        self._executor = executor

Because `futures.Executor` instances act as context managers, we should implement the same interface.

In [4]:
@patch(ConcurrentFuturesExecutor)
def __enter__(self):
    self._executor.__enter__()
    return self

In [5]:
@patch(ConcurrentFuturesExecutor)
def __exit__(self, exc_type, exc_val, exc_tb):
    self._executor.__exit__(exc_type, exc_val, exc_tb)
    return

To submit a task, we launch an `_apply_async` task which dispatches to the `futures.Executor` and chains the resulting `futures.Future` to the `aio_cf_future` object. We also chain the exceptions of the task to the result in order to propagate errors and cancellations.

In [6]:
@patch(ConcurrentFuturesExecutor)
def _apply(self, func, /, *args, **kwargs) -> asyncio.Future:
    aio_cf_future = create_future()
    # Because the unwrap stage actually needs to wait for results,
    # we create a task to do this work
    task = asyncio.create_task(self._apply_async(func, aio_cf_future, *args, **kwargs))
    # Allow task to be cancelled or raise exceptions
    chain_future_exception(task, aio_cf_future)
    return aio_cf_future

As the `futures.Executor` executor is required to resolve handles on the client, any handles provided as arguments must be resolved before dispatching to the executor. In this `_apply_async` method, we invoke the `futures.Executor.submit` method, and chain the `futures.Future` object with the given `asyncio.Future` handle. This future holds the status of the running task. 

In [7]:
@patch(ConcurrentFuturesExecutor)
async def _apply_async(self, func, aio_cf_fut: asyncio.Future, /, *args, **kwargs):
    args, kwargs = await self._process_args(args, kwargs)
    # Launch task into pool
    cf_fut = self._executor.submit(func, *args, **kwargs)
    # When we have the concurrent.futures.Future object,
    # chain it with the "proxy" fut. We can do this
    # because the data have to be retrieved locally anyway.
    asyncio.futures._chain_future(cf_fut, aio_cf_fut)

The `futures.Executor.submit` method cannot make use of `futures.Future` arguments. Instead, we have to resolve them to their computed values by waiting on them first:

In [8]:
@patch(ConcurrentFuturesExecutor)
async def _process_args(self, args, kwargs):
    # Unwrap any wrapped handles
    args = [await self._unwrap_and_wait_maybe(x) for x in args]
    kwargs = {k: await self._unwrap_and_wait_maybe(v) for k, v in kwargs.items()}
    return args, kwargs

Here, we unwrap the `asyncio.Future` handles in the arguments, and await their results so that can be passed to `futures.Executor.submit`.

In [9]:
@patch(ConcurrentFuturesExecutor)
async def _unwrap_and_wait_maybe(self, obj):
    # Ensure is wrapped
    try:
        self._unwrap_handle(obj)
    except ValueError:
        return obj

    return await self.retrieve(obj)

As the `future` that is returned by `_async` is an `asyncio.Future` object, we can use the `chain_future_handle` helper to register the necessary callbacks.

In [10]:
@patch(ConcurrentFuturesExecutor)
def _register_handle(self, handle, future):
    chain_future_handle(future, handle)

To demonstrate this, we can create a thread pool executor:

In [11]:
pool = futures.ProcessPoolExecutor()

Using this pool we can create an exectutor:

In [12]:
executor = ConcurrentFuturesExecutor(pool)

To do some work, let's implement a sleep function that returns the delay

In [13]:
def slow_function(timeout):
    time.sleep(timeout)
    return timeout

Now we can chain a few of these tasks together:

In [14]:
a = executor.submit(slow_function, 2)
b = executor.submit(slow_function, 5)
c = executor.submit(int.__add__, a, b)

We can wait for the result without retrieving its value:

In [15]:
await c

True

And when we're ready for the value, we invoke `executor.retrieve`.

In [16]:
assert await executor.retrieve(c) == 7